In [62]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from seaborn import set_style
set_style('whitegrid')

In [63]:
gross_data = pd.read_csv('https://raw.githubusercontent.com/tacookson/data/master/broadway-grosses/grosses.csv')

In [67]:
show_data = pd.read_csv('https://raw.githubusercontent.com/PSha98/Erdos-2022-project/main/Data_collection/show_info_1985-2019.csv')

In [66]:
#0) Remove shows that lasted for just 1 week?  only 20 such entries or shows with fewer than 2 performances?
#0.5) Correct Gross values based on dollar data. Use CPI data for this. 
#1) Seasonal plot for total gross, Total Seats available, Average percentage capacity across week.
#2) Trends for total gross (use average across months), (Average percentage capacity across month.)
#3) Correlation between gross and stock market data. 
#4) group data based on Show type, Theatre size, Number of weeks it ran,  and repeat steps 1,2. 



In [24]:
#0) How do we handle fewer than 2 weeks or have fewer than 4 performances. Also, How do we handle revivals? 
# Also, Some shows restart as revivals. How do we handle such shows? 

In [59]:
#0.5)  Use consumer price index to normalize dollar values. 
# CPI data

Unnamed: 0      107
Name            107
Type            107
Nominations     107
Wins            107
Awards          107
Performances    107
dtype: int64 Unnamed: 0      1400
Name            1400
Type            1400
Nominations     1400
Wins            1400
Awards          1400
Performances    1400
dtype: int64


AttributeError: 'Series' object has no attribute 'value_count'